In [2]:
from xgboost_utils import *

In [3]:
X_train, y_train, X_val, y_val = prep_data_for_xgboost()
X_train.head()

Shape of training data:  (48000, 20)
Shape of training label:  (48000, 1)
Shape of validation data:  (12000, 20)
Shape of validation label:  (12000, 1)


,rent_approval_date,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,mrts_within_3km,nearest_distance_to_mrt,pri_schs_within_6km,gep_schs_within_5km,malls_within_3km,nearest_distance_to_gep,nearest_distance_to_mall,highest_pos_corr,region_central region,region_east region,region_north region,region_north-east region,region_west region
0,2107,3,2365.067275,67.0,1979,1.366600,103.855579,11,764.670207,52,2,11,1850.376297,1034.851528,10.227563,0.0,0.0,0.0,1.0,0.0
1,2201,5,2640.896729,110.0,2003,1.401158,103.751472,4,607.646696,25,0,11,9125.370545,597.191138,11.105234,0.0,0.0,0.0,0.0,1.0
2,2107,3,2365.067275,67.0,1978,1.316994,103.768507,8,423.362498,21,4,9,860.187654,247.035245,10.227563,0.0,0.0,0.0,0.0,1.0
3,2103,3,2614.410973,74.0,1984,1.347162,103.742808,6,775.808574,39,1,7,3701.476616,787.588358,10.104055,0.0,0.0,0.0,0.0,1.0
4,2206,4,2614.410973,93.0,2002,1.450165,103.819307,4,156.306323,27,0,5,10555.854553,218.614539,11.564181,0.0,0.0,1.0,0.0,0.0


In [4]:
testing_data_raw = pd.read_csv('test.csv')

In [5]:
X_test = preprocess_xgboost(testing_data_raw, istest=True)

In [6]:
X_test.head()

,rent_approval_date,flat_type,flat_model,floor_area_sqm,lease_commence_date,latitude,longitude,mrts_within_3km,nearest_distance_to_mrt,pri_schs_within_6km,gep_schs_within_5km,malls_within_3km,nearest_distance_to_gep,nearest_distance_to_mall,highest_pos_corr,region_central region,region_east region,region_north region,region_north-east region,region_west region
0,2301,5,2640.896729,121.0,1984,1.358411,103.891722,7,821.030186,72,1,7,2454.786060,740.089998,13.204804,0.0,0.0,0.0,1.0,0.0
1,2209,4,2614.410973,100.0,1999,1.446343,103.820817,4,306.192194,27,0,7,10115.700699,254.094139,12.702486,0.0,0.0,1.0,0.0,0.0
2,2307,4,2365.067275,91.0,1980,1.305719,103.762168,5,1092.259714,18,2,6,1495.640204,497.474111,14.229989,0.0,0.0,0.0,0.0,1.0
3,2108,3,2614.410973,74.0,1986,1.344832,103.730778,7,361.691094,37,1,8,4406.337223,1593.025583,10.367904,0.0,0.0,0.0,0.0,1.0
4,2203,5,2640.896729,121.0,1983,1.345437,103.735241,6,447.607508,40,1,7,4085.891329,1454.795230,11.497292,0.0,0.0,0.0,0.0,1.0


In [8]:
X_test.shape

(30000, 20)

In [9]:
xgb_model = xgb.XGBRegressor(
            n_estimators=1800, max_depth=5, eta=0.01, subsample=0.7, colsample_bytree=0.7,
            objective="reg:squarederror"
        )
xgb_model.fit(X_train, y_train)
y_val_pred = xgb_model.predict(X_val)
print('Validation loss:', np.sqrt(mean_squared_error(y_val, y_val_pred)))
y_train_pred = xgb_model.predict(X_train)
print('Training loss:', np.sqrt(mean_squared_error(y_train, y_train_pred)))

Validation loss: 481.4906954982108
Training loss: 457.6630179309819


In [12]:
final_pred = xgb_model.predict(X_test)  
print("Length of final predictions is: ", len(final_pred))
ids = np.arange(30000)
df = pd.DataFrame({'Id': ids, 'Predicted': final_pred})
df.to_csv("kaggle_submission.csv", index=False)

Length of final predictions is:  30000
